<img src="http://www.ubu.es/sites/default/files/portal_page/images/logo_color_2l_dcha.jpg" height="200" width="200" align="right"/> 
### Author: Eduardo Tubilleja Calvo 
### Director: Álvar Arnaiz González 
### Director: Juan José Rodríguez Díez
### Title: DecisionTreeClassifier vs Ensembles

### Table of contents:
* [Select the parameters](#parameters)
* [Generate datasets](#datasets)
* [Create classifiers](#classifiers)
* [Make CrossValidation](#cross)
* [Results accuracy](#accuracy)
* [Results precision](#precision)
* [Results recall](#recall)
* [Results f1](#f1)

In thi notebook shows the comparison of the Disturbing Neighbors, Random Oracles, Rotation Forest and DecisionTreeClassifier algorithms. In which we will calculate measures to see which classifier is better in terms of precision

In [1]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_moons, make_circles, make_classification, make_multilabel_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from sklearn_ubu.disturbing_neighbors import DisturbingNeighbors
from sklearn_ubu.random_oracles import RandomOracles
from sklearn_ubu.rotation_forest import RotationForest

We create the different parameters, to which we give the desired value.

To test different sets, the characteristics and classes of each data set are varied.<a id='parameters'></a>

In [3]:
seed=1
num_samples=2500
num_datasets=5
num_features=7
num_classes=2
cross_v=5
list_tuples=[]
names_rows=[]
for i in range(num_datasets):
    num_features=num_features+3
    num_classes=num_classes+1
    list_tuples.append((num_features,num_classes))
    names_rows.append('dataset' + str(i+1) )
names_rows.append('mean')

The different data sets are generated according to the number of data sets that we have indicated.<a id='datasets'></a>

In [4]:
X=[]
y=[]
y_test=[]
for i in range(num_datasets):
    X.append(make_multilabel_classification(
        n_samples=num_samples, n_features=list_tuples[i][0], n_classes=list_tuples[i][1],  random_state=seed)[0])
    y.append(make_multilabel_classification(
        n_samples=num_samples, n_features=list_tuples[i][0], n_classes=list_tuples[i][1],  random_state=seed)[1])

Create classifiers<a id='classifiers'></a>

In [7]:
classifier=[]
classifier.append(DisturbingNeighbors(n_neighbors=20, n_features=2))
classifier.append(RandomOracles(n_oracles=5))
classifier.append(RotationForest(n_groups=5))
classifier.append(DecisionTreeClassifier())

Cross validation is calculated, with the accuracy measure, on each of the classifiers, and the average is calculated.<a id='cross'></a>

In [8]:
ensembles_accuracy=[]
ensembles_precision=[]
ensembles_recall_score=[]
ensembles_f1=[]
for i in range(len(classifier)):
    accuracy=[]
    precision=[]
    recall_score=[]
    f1=[]
    for j in range(num_datasets):
        accuracy.append(np.mean(cross_val_score(classifier[i], X[j], y[j],scoring='accuracy', cv=cross_v)))
        precision.append(np.mean(cross_val_score(classifier[i], X[j], y[j],scoring='precision_micro', cv=cross_v)))
        recall_score.append(np.mean(cross_val_score(classifier[i], X[j], y[j],scoring='recall_micro', cv=cross_v)))
        f1.append(np.mean(cross_val_score(classifier[i], X[j], y[j],scoring='f1_micro', cv=cross_v)))
        
    accuracy.append(np.mean(accuracy))
    ensembles_accuracy.append(accuracy)
    
    precision.append(np.mean(precision))
    ensembles_precision.append(precision)
    
    recall_score.append(np.mean(recall_score))
    ensembles_recall_score.append(recall_score)
    
    f1.append(np.mean(f1))
    ensembles_f1.append(f1)

In [9]:
ensembles_accuracy = np.array(ensembles_accuracy).T
ensembles_accuracy=list(ensembles_accuracy)

ensembles_precision = np.array(ensembles_precision).T
ensembles_precision=list(ensembles_precision)

ensembles_recall_score = np.array(ensembles_recall_score).T
ensembles_recall_score=list(ensembles_recall_score)

ensembles_f1 = np.array(ensembles_f1).T
ensembles_f1=list(ensembles_f1)

A test with 5 different data sets has been realized and the average of the 5 has been calculated. Thoses sets have been analyzed with the 3 algorithms realized and the DecisionTreeClassifier. 


The accuracy measure has been used. The accuracy computes subset accuracy: the set of labels predicted for a sample must exactly match the corresponding set of labels in y_true.

Below is a table to see the results:<a id='accuracy'></a>

In [10]:
names_columns=['Disturbing Neighbors', 'Random Oracles', 'Rotation Forest', 'Decision Tree Classifier']
df = pd.DataFrame(ensembles_accuracy, \
       columns=(names_columns), \
       index=(names_rows)) 
df

,Disturbing Neighbors,Random Oracles,Rotation Forest,Decision Tree Classifier
dataset1,0.70120,0.77680,0.79040,0.6848
dataset2,0.46120,0.53560,0.54080,0.4456
dataset3,0.36920,0.45720,0.46080,0.3564
dataset4,0.28320,0.38000,0.36800,0.2784
dataset5,0.30640,0.39480,0.38560,0.2968
mean,0.42424,0.50888,0.50912,0.4124


The precision measure has been used. The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.

Below is a table to see the results:<a id='precision'></a>

In [11]:
names_columns=['Disturbing Neighbors', 'Random Oracles', 'Rotation Forest', 'Decision Tree Classifier']
df = pd.DataFrame(ensembles_precision, \
       columns=(names_columns), \
       index=(names_rows)) 
df

,Disturbing Neighbors,Random Oracles,Rotation Forest,Decision Tree Classifier
dataset1,0.848033,0.879104,0.889221,0.843222
dataset2,0.740773,0.796107,0.799953,0.735488
dataset3,0.685363,0.760399,0.761327,0.682705
dataset4,0.592855,0.698554,0.714132,0.593511
dataset5,0.590578,0.695939,0.721433,0.583411
mean,0.691520,0.766021,0.777213,0.687667


The recall score measure has been used. The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

Below is a table to see the results:<a id='recall'></a>

In [12]:
names_columns=['Disturbing Neighbors', 'Random Oracles', 'Rotation Forest', 'Decision Tree Classifier']
df = pd.DataFrame(ensembles_recall_score, \
       columns=(names_columns), \
       index=(names_rows)) 
df

,Disturbing Neighbors,Random Oracles,Rotation Forest,Decision Tree Classifier
dataset1,0.862488,0.919093,0.919785,0.848409
dataset2,0.786409,0.856021,0.862141,0.761742
dataset3,0.712864,0.811667,0.824922,0.694648
dataset4,0.630400,0.705606,0.701404,0.612479
dataset5,0.627221,0.679977,0.669661,0.604858
mean,0.723877,0.794473,0.795583,0.704427


The f1 measure has been used. The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0.

Below is a table to see the results:<a id='f1'></a>

In [13]:
names_columns=['Disturbing Neighbors', 'Random Oracles', 'Rotation Forest', 'Decision Tree Classifier']
df = pd.DataFrame(ensembles_f1, \
       columns=(names_columns), \
       index=(names_rows)) 
df

,Disturbing Neighbors,Random Oracles,Rotation Forest,Decision Tree Classifier
dataset1,0.852223,0.899392,0.904484,0.847844
dataset2,0.756722,0.825919,0.828408,0.745463
dataset3,0.702520,0.784954,0.788631,0.683785
dataset4,0.614911,0.707448,0.704014,0.603844
dataset5,0.607821,0.688923,0.690003,0.592305
mean,0.706839,0.781327,0.783108,0.694648
